# 1. Import Data

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import re

In [2]:
train_path='.\\DataSet\\Train.csv'
df_train = pd.read_csv(train_path,sep='|')

test_path='.\\DataSet\\Test.csv'
df_test = pd.read_csv(test_path,sep='|')

In [3]:
df_test

,subject_Government News,subject_Middle-east,subject_News,subject_US_News,subject_left-news,subject_politics,subject_politicsNews,subject_worldnews,year,month,day,weekday,aprilfool_close,aprilfool_no,aprilfool_not sure,aprilfool_yes,title,text,label
0,0,0,0,0,0,1,0,0,2017,12,8,4,0,1,0,0,"two dead in 'day of rage' over jerusalem , pal...",jerusalem / gaza ( reuters ) - at least two pe...,1
1,1,0,0,0,0,0,0,0,2017,10,21,5,0,1,0,0,billionaire 's ano party holding big lead in c...,prague ( reuters ) - czech billionaire busines...,1
2,0,0,0,0,0,1,0,0,2017,12,4,0,0,1,0,0,the battle against islamic state,( reuters ) - it was an awkward coalition rive...,1
3,0,0,0,0,0,1,0,0,2017,11,24,4,0,1,0,0,whoa ! former democrat congresswoman reveals h...,dr . cynthia mckinney reveals hillary s murder...,0
4,0,0,0,0,0,0,1,0,2017,5,31,2,0,1,0,0,trump expected to delay u . s . embassy move t...,washington ( reuters ) - president donald trum...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17542,1,0,0,0,0,0,0,0,2017,10,22,6,0,1,0,0,japan 's abe says won 't delay tax hike unless...,tokyo ( reuters ) - japanese prime minister sh...,1
17543,0,0,0,0,1,0,0,0,2017,9,22,4,0,1,0,0,"trump kicks pro - amnesty , globalist koch bro...",president - elect donald trump told a critical...,0
17544,0,0,0,0,0,1,0,0,2017,12,12,1,0,1,0,0,"south korea 's moon , china 's xi to talk nort...",seoul / beijing ( reuters ) - curbing north ko...,1
17545,0,0,0,0,0,0,0,1,2017,12,5,1,0,1,0,0,watch liberal cnn lawyer skewer “ village idio...,NaN,0


## 1.1.Data Sample

In [4]:
# sampling if needed for shorten the test process
df_train = df_train.sample(frac=0.02, replace=True, random_state=1)
df_test = df_test.sample(frac=0.02, replace=True, random_state=1)

In [5]:
# add a mark between title and text for separating title and text when doing deep learning model
df_train['mark for title'] = 'end of title'
df_test['mark for title'] ='end of title'

In [6]:
# reassign position of a column
def reassignPos(df,ab_pos, bis_pos):
    cols = df.columns.tolist()
    col_change = cols.pop(ab_pos)
    cols.insert(bis_pos,col_change)
    df = df[cols]
    return df

In [7]:
df_train = reassignPos(df_train,19,17)
df_test = reassignPos(df_test,19,17)

In [8]:
print(len(df_train))
print(len(df_test))

1404
351


In [9]:
df_train

,subject_Government News,subject_Middle-east,subject_News,subject_US_News,subject_left-news,subject_politics,subject_politicsNews,subject_worldnews,year,month,day,weekday,aprilfool_close,aprilfool_no,aprilfool_not sure,aprilfool_yes,title,mark for title,text,label
5192,0,0,0,0,0,0,1,0,2017,9,1,4,0,1,0,0,bombshell : senate intel committee hearing rev...,end of title,while the house intelligence committee s inves...,0
50057,0,0,0,0,0,0,1,0,2017,10,16,0,0,1,0,0,paul ryan claims gop wouldn ’ t try to have de...,end of title,sometimes you get the feeling that paul ryan h...,0
21440,0,0,1,0,0,0,0,0,2016,9,23,4,0,1,0,0,long lost footage of bernie being arrested at ...,end of title,a team of documentary filmmakers made what may...,0
20609,0,0,1,0,0,0,0,0,2017,1,18,2,0,1,0,0,former israeli prime minister blasts israeli g...,end of title,it appears that the curtain that hides the tru...,0
49100,0,0,0,0,0,1,0,0,2016,4,14,3,0,1,0,0,house speaker ryan backs continued aid to egypt,end of title,washington ( reuters ) - u . s . house of repr...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63104,0,0,0,0,0,0,0,1,2017,12,13,2,0,1,0,0,pakistani court summons police in case of miss...,end of title,"lahore , pakistan ( reuters ) - a pakistani co...",1
2446,0,0,0,0,0,0,1,0,2017,11,9,3,0,1,0,0,trump gets humiliated by his own joint chiefs ...,end of title,it appears donald trump jumped the gun when he...,0
48846,1,0,0,0,0,0,0,0,2017,10,27,4,0,1,0,0,italian man sentenced to 24 years for infectin...,end of title,rome ( reuters ) - an italian man was sentence...,1
7393,0,0,1,0,0,0,0,0,2017,6,4,6,0,1,0,0,trump responds to uk attack with apparent crit...,end of title,london ( reuters ) - u . s . president donald ...,1


## 1.2. Separation for X and Y

In [10]:
def getXY(df):
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    return X,y

In [11]:
X_train,y_train = getXY(df_train)
X_test, y_test = getXY(df_test)

In [12]:
len(X_test[0])

19

In [22]:
X_test[10][17]

'end of title'

In [13]:
len(X_test)

351

In [14]:
X_test

array([[0, 0, 0, ...,
        'florida rejected anti - terror bill ahead of attack in orlando',
        'end of title',
        'oh no ! florida killed a counter - terrorism bill right before the orlando attack . florida lawmakers rejected a key piece of counterterrorism legislation earlier this year , prompting outrage from some defense experts who told the washington free beacon that the victims of sunday s mass terror attack in orlando could have used the measure to hold those who plotted the assault accountable . on the heels of sunday s shooting , in which an attacker who pledged allegiance to the isis terror group killed 49 bystanders , marking the deadliest terror attack on u . s . soil since 9 / 11 , some are criticizing florida s legislature for rejecting a counter - terrorism bill that would have paved the way for victims of these crimes to sue the organizers and sponsors in state court . the legislation , known as andy s law , has already passed in at least five states but w

In [23]:
y_test

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,

# 2. Vectorization
## 2.1. Import GloVe pre-trained Data

In [24]:
# import learned embedding resource glove pretrained data
GloVe_path=".\DataSet\glove.840B.300d.txt"

In [25]:
# function for loading GloVe
def load_glove(path):
    glove_dict = dict()
    words = list()
    with open(path, mode='r', encoding="utf-8") as vec_file:
        for line in tqdm(vec_file):
            values = line.split()
            word = values[0]
            vec = np.array(values[1:])
            glove_dict[word] = vec

    print("There are {0} distinct word vectors in this pretrained GloVe DataSet.".format(len(glove_dict)))
    return glove_dict

In [26]:
glove_dict = load_glove(GloVe_path)

2196017it [01:59, 18349.01it/s]

There are 2195884 distinct word vectors in this pretrained GloVe DataSet.


In [48]:
# function for get vectorization of sentence and output as matrix
def getVec(sentence):
    'function for getting vector of the sentence'
    try:
        sentence = re.sub(r' +',r' ',sentence)
        word_list=sentence.split()
        sentence_matrix = list()
        for word in word_list:
            try:
                vec = glove_dict[word]
                vec = np.array(vec).astype(np.float)
                sentence_matrix.append(vec)
            except:
                pass
        sentence_matrix = np.array(sentence_matrix)
        
    except TypeError:# if there is np.nan value
        sentence_matrix = np.zeros((1,300))
    return sentence_matrix

In [49]:
# test
sen1="i like milk"
sen2='ok'
ma1=getVec(sen1)
ma2=getVec(sen2)
ma3 = np.append(ma1,ma2,0)
print(len(ma3))
ma3

4


array([[ 0.18733 ,  0.40595 , -0.51174 , ...,  0.16495 ,  0.18757 ,
         0.53874 ],
       [-0.18417 ,  0.055115, -0.36953 , ..., -0.23808 ,  0.37132 ,
         0.36197 ],
       [-0.62388 ,  0.21805 ,  0.29327 , ..., -0.96244 ,  0.28478 ,
         0.27701 ],
       [-0.13199 ,  0.15186 , -0.65313 , ..., -0.33259 , -0.17585 ,
         0.12933 ]])

## 2.2. Vectorization for Deep Learning

In [55]:
# function for vectorization of X
def vectorization(X):
    X_out = list()
    for eachRow in tqdm(X):
        text_row = ''
        for eachEle in eachRow:# concatenate the attributes into one sentence
            text_row = text_row +" " +str(eachEle)
        
        text_row = text_row.strip()
        text_matrix = getVec(text_row)
        X_out.append(text_matrix)
    X_out = np.array(X_out)
    return X_out

In [56]:
X_test_vec = vectorization(X_test)

100%|████████████████████████████████████████████████████████████████████████████████| 351/351 [00:21<00:00, 16.54it/s]


In [23]:
X_train_vec = vectorization(X_train)

100%|██████████████████████████████████████████████████████████████████████████████| 1404/1404 [01:30<00:00, 15.48it/s]


In [57]:
print(len(X_test_vec))
print(len(X_train_vec))

351


NameError: name 'X_train_vec' is not defined

In [25]:
X_test_vec[0][0]

array([-2.9710e-01,  5.2555e-01, -1.1982e-01,  2.2320e-01, -2.9821e-01,
       -1.4652e-01, -3.5873e-01, -5.6424e-01,  4.5164e-01,  3.2841e-01,
        2.9934e-01,  2.4738e-02, -8.8498e-01, -4.7742e-03, -4.1028e-01,
        3.1131e-02, -9.5197e-02,  1.9017e+00, -6.3973e-01, -2.5414e-01,
       -2.2079e-02,  4.3251e-02, -1.7333e-01,  6.8227e-02,  2.0963e-01,
       -5.2900e-01,  8.8352e-02,  2.1385e-01,  1.8160e-01,  3.4159e-01,
        2.5816e-01, -4.0376e-01,  2.9269e-02, -2.6315e-01,  7.9829e-01,
        1.7313e-01,  2.2382e-01,  4.1699e-01, -1.3711e-01,  7.7650e-02,
       -2.6138e-02, -5.7187e-01,  4.0936e-01,  4.6736e-02, -1.8373e-01,
       -2.2086e-01, -1.1758e-01,  1.1521e+00, -1.9167e-01, -3.6863e-02,
       -2.5841e-01,  2.1220e-01, -1.7783e-01, -8.1226e-01,  2.6196e-01,
        2.6904e-01, -1.7779e-01,  5.6400e-01,  3.9413e-01, -1.2212e-01,
        3.2701e-01, -2.7895e-01,  1.3530e-01, -1.2869e-01, -2.7203e-01,
       -1.3178e-01,  6.3718e-01, -1.7704e-01,  3.4271e-01, -5.40

## 2.3. Vectorization for Machine Learning

In [51]:
# function for vectorization only the text part of X, and use the average vector as the new attributes to replace the text column
def vectorizationPart(X,pos_title=16,pos_text=18):
    X_out = list()
    for eachRow in tqdm(X):
        list_row=list()
        for i in range(pos_title):
            # make categorical attributes
            list_row.append(eachRow[i])
        #print(eachRow[pos_title])
        #try:
        title_vec = getVec(eachRow[pos_title])
        title_vec_mean = list(np.mean(title_vec,axis=0))
        #except:
        #    title_vec_mean = np.zeros((1,300))
        #    title_vec_mean = list(title_vec_mean[0])
        list_row = list_row + title_vec_mean
        #print(eachRow[pos_text])
        #try:
        text_vec = getVec(eachRow[pos_text])
        text_vec_mean = list(np.mean(text_vec,axis=0))
            
        #except:
        #    text_vec_mean = np.zeros((1,300))
        #    text_vec_mean = list(title_vec_mean[0])
        list_row = list_row + text_vec_mean
        
        list_row = np.array(list_row)

            
        X_out.append(list_row)
    X_out = np.array(X_out)
    return X_out

In [52]:
X_test_cols = vectorizationPart(X_test)
X_train_cols = vectorizationPart(X_train)

100%|██████████████████████████████████████████████████████████████████████████████| 1404/1404 [01:25<00:00, 16.44it/s]


In [60]:
X_test_cols

array([[ 0.        ,  0.        ,  0.        , ..., -0.10328256,
         0.02047796,  0.05256463],
       [ 0.        ,  1.        ,  0.        , ..., -0.08553779,
        -0.09282349,  0.06472201],
       [ 0.        ,  0.        ,  0.        , ..., -0.11412325,
        -0.0410173 ,  0.05300926],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -0.09853726,
         0.01338801,  0.03899308],
       [ 0.        ,  0.        ,  0.        , ..., -0.10059129,
         0.01005148,  0.02113287],
       [ 0.        ,  0.        ,  1.        , ..., -0.09528402,
         0.03270608,  0.0228382 ]])

# 3. Storage

In [61]:
# function for saving as pickle file
def saveData(name,data):
    pickle_file = open('DataSet\\'+name,mode='wb')
    pickle.dump(data,pickle_file)
    pickle_file.close()

In [27]:
saveData('y_train.pkl',y_train)
saveData('y_test.pkl',y_test)
saveData('X_train_vec.pkl',X_train_vec)
saveData('X_test_vec.pkl',X_test_vec)

In [62]:
saveData('X_train_cols.pkl',X_train_cols)
saveData('X_test_cols.pkl',X_test_cols)